In [1]:
from IPython.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))
from elasticsearch import Elasticsearch 
from elasticsearch import helpers
from sklearn.cluster import KMeans
import requests
from urllib.parse import urljoin
from time import gmtime, strftime
from itertools import product
import warnings
from scipy import stats
import chart_studio.plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline
import pandas as pd
import numpy as np
import datetime
from os.path import dirname
from os import listdir

from sqlalchemy import create_engine, MetaData
from pandas import read_sql
from os.path import join, dirname, abspath, exists

In [2]:
es=Elasticsearch([{'host':'localhost','port':9200}], timeout=300, max_retries=10)

In [5]:
es.indices.delete(index='downloads', ignore=[400, 404])

{'acknowledged': True}

In [ ]:
from exploratory_analysis.__init__ import create_exploratory_analyse

In [3]:
from customeranalytics.ml_process.__init__ import create_ml

/Users/caglanakpinar/miniforge3/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:143: DeprecationWarning:

plotly.graph_objs.Annotation is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Annotation
  - plotly.graph_objs.layout.scene.Annotation




[Errno 21] Is a directory: '/Users/caglanakpinar/Desktop/temp_customer_analytics_folder/build_in_reports/main/2021-06-19'


# Exploratory analysis

In [ ]:
datetime.datetime.now().isoformat()

In [ ]:
ea_configs = {"date": None,
              "funnel": {"actions": [],
                         "purchase_actions": ["has_basket", "order_screen"],
                         "host": 'localhost',
                         "port": '9200',
                         'download_index': 'location_1',
                         'order_index': 'location_1'},
              "cohort": {"has_download": True, "host": 'localhost', "port": '9200',
                         'download_index': 'location_1', 'order_index': 'location_1'},
              "products": {"has_product_connection": True, "host": 'localhost', "port": '9200', 'download_index': 'downloads', 'order_index': 'orders'},
              "rfm": {"host": 'localhost', "port": '9200', 'download_index': 'downloads', 'order_index': 'location_2'},
              "stats": {"host": 'localhost', "port": '9200', 'download_index': 'downloads', 'order_index': 'location_3'}
             }
create_exploratory_analyse(ea_configs, 'stats')

In [ ]:
create_exploratory_analysis(ea_configs)

In [5]:
es.search(index='orders', body={'size': 100, 'from': 0})['hits']['hits']

[{'_index': 'orders',
  '_type': '_doc',
  '_id': '97682578',
  '_score': 1.0,
  '_source': {'id': '97682578',
   'date': '2021-03-31T17:32:00',
   'actions': {'has_basket': True,
    'order_screen': True,
    'purchased': True,
    'has_sessions': True},
   'client': 'u_u_1828393',
   'promotion_id': None,
   'payment_amount': 32.75,
   'discount_amount': 0.0,
   'basket': {'p_54': {'price': 12.0, 'category': 'p_c_9'},
    'p_9': {'price': 4.48, 'category': 'p_c_6'},
    'p_198': {'price': 2.655, 'category': 'p_c_5'},
    'p_83': {'price': 1.68, 'category': 'p_c_3'},
    'p_132': {'price': 7.455, 'category': 'p_c_9'},
    'p_73': {'price': 4.48, 'category': 'p_c_8'}},
   'total_products': 6.0,
   'session_start_date': '2021-03-31T17:21:55',
   'dimension': 'location_1'}},
 {'_index': 'orders',
  '_type': '_doc',
  '_id': '29886487',
  '_score': 1.0,
  '_source': {'id': '29886487',
   'date': '2021-03-31T15:15:00',
   'actions': {'has_basket': True,
    'order_screen': True,
    'purch

In [ ]:
match = {'size': 10, 'from': 0, '_source': True, 'fields': ['id', 'session_start_date', 'basket'], 'query': {'bool': {'must': [{'term': {'actions.purchased': True}}]}}}

In [ ]:
es.search(index='orders', body=match)['hits']['hits']

### ml works

In [7]:
ml_configs = {"date": None,
              'time_period': '6 months',
              "segmentation": {"host": 'localhost', "port": '9200',
                               'download_index': 'downloads', 'order_index': 'orders'},
              "clv_prediction": {"temporary_export_path": "/Users/caglanakpinar/Desktop/temp_customer_analytics_folder_2/",
                                 "host": 'localhost', "port": '9200',
                                 'download_index': 'downloads', 'order_index': 'orders', 'time_period': '6 months'},
              "abtest": {"has_product_connection": False,
                         "has_promotion_connection": False, "temporary_export_path": "/Users/caglanakpinar/Desktop/temp_customer_analytics_folder/",
                         "host": 'localhost', "port": '9200', 'download_index': 'downloads', 'order_index': 'location_3'},
              "anomaly": {"host": 'localhost', "port": '9200',
                          'download_index': 'downloads', 'order_index': 'orders'},
          }
create_ml(ml_configs, 'clv_prediction')

*****  CLV Prediction  *****
db connection is done!
received : {'job': 'train_prediction', 'order_count': None, 'customer_indicator': 'client', 'amount_indicator': 'payment_amount', 'data_source': 'csv', 'data_query_path': '/Users/caglanakpinar/Desktop/temp_customer_analytics_folder_2/temp_data.csv', 'time_period': '6*month', 'time_indicator': 'session_start_date', 'export_path': '/Users/caglanakpinar/Desktop/temp_customer_analytics_folder_2/'}
    session_start_date       client  payment_amount   dimension
0  2021-03-31 17:21:55  u_u_1828393          32.750  location_1
1  2021-03-31 15:09:13  u_u_1429801          13.630  location_1
2  2021-03-31 09:59:40  u_u_1872775          36.970  location_1
3  2021-03-31 14:22:51  u_u_1662030          54.720  location_1
4  2021-03-31 13:59:47  u_u_1580295           9.935  location_1
data size : 359317
previous last model trained : 20210613 , for  trained_next_purchase_model
***** Next purchase train model process  *****
Previous model already exit

4  2021-03-31 13:59:47  u_u_1580295           9.935  location_1
data size : 359317
previous last model trained : 20210613 , for  trained_newcomers_model
***** Newcomer CLV Prediction train model process  *****
Previous model already exits in arrange__data_for_model the given directory  '/Users/caglanakpinar/Desktop/temp_customer_analytics_folder_2/'.
***** PREDICTION *****
date : 2021-06-20 00:00:00
date : 2021-06-21 00:00:00
date : 2021-06-22 00:00:00
date : 2021-06-23 00:00:00
date : 2021-06-24 00:00:00
date : 2021-06-25 00:00:00
date : 2021-06-26 00:00:00
date : 2021-06-27 00:00:00
date : 2021-06-28 00:00:00
date : 2021-06-29 00:00:00
date : 2021-06-30 00:00:00
date : 2021-07-01 00:00:00
date : 2021-07-02 00:00:00
date : 2021-07-03 00:00:00
date : 2021-07-04 00:00:00
date : 2021-07-05 00:00:00
date : 2021-07-06 00:00:00
date : 2021-07-07 00:00:00
date : 2021-07-08 00:00:00
date : 2021-07-09 00:00:00
date : 2021-07-10 00:00:00
date : 2021-07-11 00:00:00
date : 2021-07-12 00:00:00
dat

date : 2021-07-31 00:00:00
date : 2021-08-01 00:00:00
date : 2021-08-02 00:00:00
date : 2021-08-03 00:00:00
date : 2021-08-04 00:00:00
date : 2021-08-05 00:00:00
date : 2021-08-06 00:00:00
date : 2021-08-07 00:00:00
date : 2021-08-08 00:00:00
date : 2021-08-09 00:00:00
date : 2021-08-10 00:00:00
date : 2021-08-11 00:00:00
date : 2021-08-12 00:00:00
date : 2021-08-13 00:00:00
date : 2021-08-14 00:00:00
date : 2021-08-15 00:00:00
date : 2021-08-16 00:00:00
date : 2021-08-17 00:00:00
date : 2021-08-18 00:00:00
date : 2021-08-19 00:00:00
date : 2021-08-20 00:00:00
date : 2021-08-21 00:00:00
date : 2021-08-22 00:00:00
date : 2021-08-23 00:00:00
date : 2021-08-24 00:00:00
date : 2021-08-25 00:00:00
date : 2021-08-26 00:00:00
date : 2021-08-27 00:00:00
date : 2021-08-28 00:00:00
date : 2021-08-29 00:00:00
date : 2021-08-30 00:00:00
date : 2021-08-31 00:00:00
date : 2021-09-01 00:00:00
date : 2021-09-02 00:00:00
date : 2021-09-03 00:00:00
date : 2021-09-04 00:00:00
date : 2021-09-05 00:00:00
d

date : 2021-09-13 00:00:00
date : 2021-09-14 00:00:00
date : 2021-09-15 00:00:00
date : 2021-09-16 00:00:00
date : 2021-09-17 00:00:00
date : 2021-09-18 00:00:00
date : 2021-09-19 00:00:00
date : 2021-09-20 00:00:00
date : 2021-09-21 00:00:00
date : 2021-09-22 00:00:00
date : 2021-09-23 00:00:00
date : 2021-09-24 00:00:00
date : 2021-09-25 00:00:00
date : 2021-09-26 00:00:00
date : 2021-09-27 00:00:00
date : 2021-09-28 00:00:00
date : 2021-09-29 00:00:00
date : 2021-09-30 00:00:00
date : 2021-10-01 00:00:00
date : 2021-10-02 00:00:00
date : 2021-10-03 00:00:00
date : 2021-10-04 00:00:00
date : 2021-10-05 00:00:00
date : 2021-10-06 00:00:00
date : 2021-10-07 00:00:00
date : 2021-10-08 00:00:00
date : 2021-10-09 00:00:00
date : 2021-10-10 00:00:00
date : 2021-10-11 00:00:00
date : 2021-10-12 00:00:00
date : 2021-10-13 00:00:00
date : 2021-10-14 00:00:00
date : 2021-10-15 00:00:00
date : 2021-10-16 00:00:00
date : 2021-10-17 00:00:00
date : 2021-10-18 00:00:00
date : 2021-10-19 00:00:00
d

date : 2021-10-27 00:00:00
date : 2021-10-28 00:00:00
date : 2021-10-29 00:00:00
date : 2021-10-30 00:00:00
date : 2021-10-31 00:00:00
date : 2021-11-01 00:00:00
date : 2021-11-02 00:00:00
date : 2021-11-03 00:00:00
date : 2021-11-04 00:00:00
date : 2021-11-05 00:00:00
date : 2021-11-06 00:00:00
date : 2021-11-07 00:00:00
date : 2021-11-08 00:00:00
date : 2021-11-09 00:00:00
date : 2021-11-10 00:00:00
date : 2021-11-11 00:00:00
date : 2021-11-12 00:00:00
date : 2021-11-13 00:00:00
date : 2021-11-14 00:00:00
date : 2021-11-15 00:00:00
date : 2021-11-16 00:00:00
date : 2021-11-17 00:00:00
date : 2021-11-18 00:00:00
date : 2021-11-19 00:00:00
date : 2021-11-20 00:00:00
date : 2021-11-21 00:00:00
date : 2021-11-22 00:00:00
date : 2021-11-23 00:00:00
date : 2021-11-24 00:00:00
date : 2021-11-25 00:00:00
date : 2021-11-26 00:00:00
date : 2021-11-27 00:00:00
date : 2021-11-28 00:00:00
date : 2021-11-29 00:00:00
date : 2021-11-30 00:00:00
date : 2021-12-01 00:00:00
date : 2021-12-02 00:00:00
d

bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0.05  MB
bulk size : 10000
memory usage : 0

In [ ]:
pd.read_csv("/Users/mac/Downloads/es-7.10.0/results_week_20210603_20210529.csv")

In [ ]:
data = pd.DataFrame([i for i in range(100000)])
sum(data.memory_usage()) / 8000000

In [ ]:
match = {'size': 1, 'from': 0, '_source': True, 'fields': ['id', 'client', 'basket', 'session_start_date', 'payment_amount'], 'query': {'bool': {'must': [{'term': {'actions.has_basket': True}}, {'range': {'session_start_date': {'lt': '2021-04-03'}}}]}}}

In [ ]:
es.search(index='orders', body=match)['hits']['hits'][0]['_source']['basket']

In [16]:

res = []
for r in es.search(index='reports', body={"size": 1000, "from": 0})['hits']['hits']:
    try:
        print(r['_source'].keys())
        res.append({'report_name': r['_source']['report_name'],
                    'report_date': r['_source']['report_date'],
                    'time_period': r['_source']['report_types'].get('time_period', None),
                    'type': r['_source']['report_types'].get('type', None),
                    '_from': r['_source']['report_types'].get('from', None),
                    'partition': r['_source']['report_types'].get('partition', None),
                    '_to': r['_source']['report_types'].get('to', None),
                    'abtest_type': r['_source']['report_types'].get('abtest_type', None),
                    'report_types': r['_source']['report_types'],
                    'index': r['_source']['index'],
                    'data': r['_source']['data']})
    except Exception as e:
        print(e)

dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_dat

dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_date', 'report_name', 'index', 'report_types', 'data'])
dict_keys(['id', 'report_dat

In [ ]:
pd.DataFrame(res)

In [17]:
pd.DataFrame(res).query("report_name == 'clv_prediction'")

,report_name,report_date,time_period,type,_from,partition,_to,abtest_type,report_types,index,data
0,clv_prediction,2021-06-26T00:00:00,6 months,None,None,29,None,None,"{'time_period': '6 months', 'partition': 29}",main,"[{'dimension': 'location_3', 'date': '2021-04-..."
1,clv_prediction,2021-06-26T00:00:00,6 months,None,None,32,None,None,"{'time_period': '6 months', 'partition': 32}",main,"[{'dimension': 'location_3', 'date': '2021-04-..."
2,clv_prediction,2021-06-26T00:00:00,6 months,None,None,33,None,None,"{'time_period': '6 months', 'partition': 33}",main,"[{'dimension': 'location_3', 'date': '2021-04-..."
3,clv_prediction,2021-06-26T00:00:00,6 months,None,None,11,None,None,"{'time_period': '6 months', 'partition': 11}",main,"[{'dimension': 'location_3', 'date': '2021-04-..."
4,clv_prediction,2021-06-26T00:00:00,6 months,None,None,12,None,None,"{'time_period': '6 months', 'partition': 12}",main,"[{'dimension': 'location_2', 'date': '2021-04-..."
...,...,...,...,...,...,...,...,...,...,...,...
237,clv_prediction,2021-06-26T00:00:00,6 months,None,None,231,None,None,"{'time_period': '6 months', 'partition': 231}",main,"[{'dimension': 'location_3', 'date': '2021-06-..."
238,clv_prediction,2021-06-26T00:00:00,6 months,None,None,232,None,None,"{'time_period': '6 months', 'partition': 232}",main,"[{'dimension': 'location_2', 'date': '2021-06-..."
239,clv_prediction,2021-06-26T00:00:00,6 months,None,None,235,None,None,"{'time_period': '6 months', 'partition': 235}",main,"[{'dimension': 'location_2', 'date': '2021-06-..."
240,clv_prediction,2021-06-26T00:00:00,6 months,None,None,238,None,None,"{'time_period': '6 months', 'partition': 238}",main,"[{'dimension': 'location_2', 'date': '2021-11-..."


In [ ]:
pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'clv_prediction'")['data'])[-1]).query("client  == 'newcomers'")

In [ ]:
pd.DataFrame(res)['report_name'].unique()

In [ ]:
pd.DataFrame(res).query("report_name == 'anomaly'")

In [ ]:
pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'anomaly' and type == 'daily_funnel'")['data'])[0])

In [ ]:
pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'cohort' and time_period == 'daily' and type == 'downloads'")['data'])[0])

In [ ]:
pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'funnel' and time_period == 'daily' and type == 'downloads'"
                                         )['data'])[0]).to_csv("exploratory_analysis/sample_data/sample_data_daily_funnel_downloads.csv", index=False)

In [ ]:
pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'funnel' and time_period == 'hourly' and type == 'downloads'"
                                         )['data'])[0]).to_csv("exploratory_analysis/sample_data/sample_data_hourly_funnel_downloads.csv", index=False)

In [ ]:
pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'cohort' and time_period == 'daily' and type == 'orders' and to == 1")['data'])[0]).to_csv(
    "exploratory_analysis/sample_data/sample_data_weekly_cohort_downloads.csv", index=False)

In [ ]:
pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'cohort' and time_period == 'daily' and type == 'orders' and to == '2.0'")['data'])[0])

In [ ]:
pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'cohort' and time_period == 'weekly' and type == 'downloads' and to == '1.0'")['data'])[0]).to_csv(
    "exploratory_analysis/sample_data/sample_data_weekly_cohort_downloads.csv", index=False)

In [ ]:
['daily_cohort_downloads', 'daily_cohort_from_1_to_2', 'daily_cohort_from_2_to_3', 'daily_cohort_from_3_to_4',
                              'weekly_cohort_downloads', 'weekly_cohort_from_1_to_2', 'weekly_cohort_from_2_to_3', 'weekly_cohort_from_3_to_4']

In [ ]:
pd.DataFrame(res).query("report_name == 'stats'")

In [ ]:
def convert_to_date(date):
    """
    Convert str date to datetime. If is NULL skip the process and return None.
    :param date: str format; %Y-%m-%d %H:%M:%S
    :return:
    """
    if date == date:
        return datetime.datetime.strptime(str(date)[0:10] + ' ' + str(date)[11:19], "%Y-%m-%d %H:%M:%S")
    else:
        return None

In [ ]:
match = {
   "size": 1,
   "sort": { "session_start_date": "desc"},

}
convert_to_date(es.search(index='orders', body={"size": 1, "sort": { "session_start_date": "desc"}})['hits']['hits'][0]['_source']['session_start_date'])

In [ ]:
match2 = {
"size": 0,
"aggs" : {
    "langs" : {
        "terms" : { "field" : "dimension",  "size" : 500 }
    }
}}

es.search(index='orders', body=match2)['hits']['hits']

In [ ]:
[r['fields']['dimension'][0] for r in es.search(index='orders', body={"fields": ["dimension"]})['hits']['hits']]

In [ ]:
data = pd.read_csv(join(abspath(""), "exploratory_analysis", "sample_data", "sample_data_hourly_orders.csv")).drop('Unnamed: 0', axis=1).rename(columns={"hours": "hourly"})
data.to_csv(join(abspath(""), "exploratory_analysis", "sample_data", "sample_data_hourly_orders.csv"), index=False)

In [ ]:
data = pd.read_csv(join(abspath(""), "exploratory_analysis", "sample_data", "sample_data_weekly_orders.csv")).drop('Unnamed: 0', axis=1).rename(columns={"date": "weekly"})
data.to_csv(join(abspath(""), "exploratory_analysis", "sample_data", "sample_data_weekly_orders.csv"), index=False)

In [ ]:
data = pd.read_csv(join(abspath(""), "exploratory_analysis", "sample_data", "sample_data_monthly_orders.csv")).drop('Unnamed: 0', axis=1).rename(columns={"date": "monthly"})
data.to_csv(join(abspath(""), "exploratory_analysis", "sample_data", "sample_data_monthly_orders.csv"), index=False)

In [ ]:
data = pd.read_csv(join(abspath(""), "exploratory_analysis", "sample_data", "sample_data_daily_orders.csv")).drop('Unnamed: 0', axis=1).rename(columns={"date": "daily"})
data.to_csv(join(abspath(""), "exploratory_analysis", "sample_data", "sample_data_daily_orders.csv"), index=False)

In [ ]:
weekly_average_session_per_user = pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'stats' and type == 'weekly_average_session_per_user'")['data'])[-1])
weekly_average_order_per_user = pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'stats' and type == 'weekly_average_order_per_user'")['data'])[-1])
purchase_amount_distribution = pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'stats' and type == 'purchase_amount_distribution'")['data'])[-1])
weekly_average_payment_amount = pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'stats' and type == 'weekly_average_payment_amount'")['data'])[-1])

In [ ]:
dfs = zip(["weekly_average_session_per_user", "weekly_average_order_per_user", "purchase_amount_distribution", "weekly_average_payment_amount"], 
          [weekly_average_session_per_user, weekly_average_order_per_user, purchase_amount_distribution, weekly_average_payment_amount])
for df in dfs:
    df[1].to_csv(join(abspath(""), "exploratory_analysis", "sample_data", df[0] + ".csv"), index=False)

In [ ]:
purchase_amount_distribution['payment_bins'] = purchase_amount_distribution['payment_bins'].apply(lambda x: float(x))
purchase_amount_distribution['orders'] = purchase_amount_distribution['orders'].apply(lambda x: int(x))

In [ ]:
purchase_amount_distribution

In [ ]:
pd.DataFrame(res).query("report_name == 'stats'")['type'].unique()

In [ ]:
purchase_amount_distribution['orders'] = purchase_amount_distribution['orders'].apply(lambda x: int(x))
purchase_amount_distribution = purchase_amount_distribution.sort_values(by='payment_bins', ascending=True)

In [ ]:


purchase_amount_distribution['payment_bins'] = purchase_amount_distribution['payment_bins'].apply(lambda x: float(x))
purchase_amount_distribution['orders'] = purchase_amount_distribution['orders'].apply(lambda x: int(x))
purchase_amount_distribution = purchase_amount_distribution.sort_values(by='payment_bins', ascending=True)
trace = []
for _bin in purchase_amount_distribution.to_dict('results'):
    print(_bin['payment_bins'], _bin['orders'])
    trace.append(go.Bar(x=[_bin['payment_bins']], y=[_bin['orders']]))

In [ ]:
import plotly.graph_objects as go
animals=['giraffes', 'orangutans', 'monkeys']

fig = go.Figure(data=trace)
# Change the bar mode
fig.update_layout(barmode='group', color='blue')
fig.show()

In [ ]:
[dict(i)['abtest_type'] for i in list(pd.DataFrame(res).query("report_name == 'abtest'")['report_types'])]

In [ ]:
abtests = pd.DataFrame(res).query("report_name == 'abtest'")
abtests

In [ ]:
ab_test_values = {'promotion_usage_before_after_orders': pd.DataFrame(),
                  'product_usage_before_after_amount': pd.DataFrame(),
                  'segments_change_weekly_before_after_orders': pd.DataFrame(),
                  'segments_change_monthly_before_after_orders': pd.DataFrame(),
                  'promotion_usage_before_after_amount': pd.DataFrame(),
                  'product_usage_before_after_orders': pd.DataFrame(),
                  'segments_change_daily_before_after_orders': pd.DataFrame(),
                  'segments_change_daily_before_after_amount': pd.DataFrame(),
                  'segments_change_weekly_before_after_amount': pd.DataFrame(),
                  'segments_change_monthly_before_after_amount': pd.DataFrame(),
                  'promotion_comparison': pd.DataFrame()}

for i in pd.DataFrame(res).query("report_name == 'abtest'").to_dict('results'):
    ab_test_values[i['report_types']['abtest_type']] = pd.DataFrame(i['data'])


In [ ]:
ab_test_values['promotion_usage_before_after_orders']['diff'] = ab_test_values['promotion_usage_before_after_orders']['mean_validation'] - ab_test_values[
    'promotion_usage_before_after_orders']['mean_control']

### Before After Customers of Average Purchase Count Test (Test Rejected!)
    - Assumption "Promotion of usage increase the purchase count of each user?"
    - This graph shows the promotion which we tested and rejected of the Assumption.
    - Related the customers who has used promotions, This is the A/B Test reults of these customers of before order count of each customer and after order count of each result.
    - Graph shows the customers of the average purchase count of  of before and after the usage of each promotion.

In [ ]:
_abtest = ab_test_values['promotion_usage_before_after_orders'].query("is_orders_increased_per_promotions == False").sort_values('diff')
trace = [go.Bar(name="before avereage order count per customers", x=_abtest['promotions'], y=_abtest['mean_control'], text=_abtest['diff']),
         go.Bar(name="after avereage order count per customers", x=_abtest['promotions'], y=_abtest['mean_validation'], text=_abtest['diff'])
         ]
fig = go.Figure(data=trace, layout_yaxis_range=[round(max(0, min(min(_abtest['mean_control']), min(_abtest['mean_validation'])) - 0.02), 2), 
                                               round(max(0, max(max(_abtest['mean_control']), max(_abtest['mean_validation'])) - 0.02), 2)])
fig.update_layout(barmode='group', xaxis_tickangle=-45, )
# Change the bar mode
fig.show()

In [ ]:
_abtest.to_csv(join(abspath(""), "exploratory_analysis", "sample_data", "promotion_usage_before_after_orders_accept.csv"), index=False)

### Before After Customers of Average Purchase Count Test (Test Accepted!)
    - Assumption "Promotion of usage increase the purchase count of each user?"
    - This graph shows the promotion which we tested and Accepted of the Assumption.
    - Related the customers who has used promotions, This is the A/B Test reults of these customers of before order count of each customer and after order count of each result.
    - Graph shows the customers of the average purchase count of  of before and after the usage of each promotion.

In [ ]:
_abtest = ab_test_values['promotion_usage_before_after_orders'].query("is_orders_increased_per_promotions == True")
trace = [go.Bar(name="before avereage order count per customers", x=_abtest['promotions'], y=_abtest['mean_control']),
         go.Bar(name="after avereage order count per customers", x=_abtest['promotions'], y=_abtest['mean_validation'])
         ]
fig = go.Figure(data=trace, layout_yaxis_range=[round(max(0, min(min(_abtest['mean_control']), min(_abtest['mean_validation'])) - 0.02), 2), 
                                               round(max(0, max(max(_abtest['mean_control']), max(_abtest['mean_validation'])) - 0.02), 2)])
fig.update_layout(barmode='group', xaxis_tickangle=-45)

# Change the bar mode
fig.show()

In [ ]:
_abtest.to_csv(join(abspath(""), "exploratory_analysis", "sample_data", "promotion_usage_before_after_orders_reject.csv"), index=False)

### Before After Customers of Average Purchase Amount Test (Test Rejected!)
    - Assumption "Promotion of usage increase the purchase count of each user?"
    - This graph shows the promotion which we tested and rejected of the Assumption.
    - Related the customers who has used promotions, This is the A/B Test reults of these customers of before order count of each customer and after order count of each result.
    - Graph shows the customers of the average purchase count of  of before and after the usage of each promotion.

In [ ]:
ab_test_values['promotion_usage_before_after_amount']['diff'] = ab_test_values['promotion_usage_before_after_amount']['mean_validation'] - ab_test_values[
    'promotion_usage_before_after_amount']['mean_control']

In [ ]:
_abtest = ab_test_values['promotion_usage_before_after_amount'].query("is_amount_increased_per_promotions == False").sort_values('diff')
trace = [go.Bar(name="before avereage purchase amount per customers", x=_abtest['promotions'], y=_abtest['mean_control'], text=_abtest['diff']),
         go.Bar(name="after avereage purchase amount per customers", x=_abtest['promotions'], y=_abtest['mean_validation'], text=_abtest['diff'])
         ]
fig = go.Figure(data=trace, layout_yaxis_range=[round(max(0, min(min(_abtest['mean_control']), min(_abtest['mean_validation'])) - 0.02), 2), 
                                               round(max(0, max(max(_abtest['mean_control']), max(_abtest['mean_validation'])) - 0.02), 2)])
fig.update_layout(barmode='group', xaxis_tickangle=-45, )
# Change the bar mode
fig.show()

In [ ]:
_abtest.to_csv(join(abspath(""), "exploratory_analysis", "sample_data", "promotion_usage_before_after_amount_reject.csv"), index=False)

### Before After Customers of Average Purchase Count Test (Test Accepted!)
    - Assumption "Promotion of usage increase the purchase count of each user?"
    - This graph shows the promotion which we tested and Accepted of the Assumption.
    - Related the customers who has used promotions, This is the A/B Test reults of these customers of before order count of each customer and after order count of each result.
    - Graph shows the customers of the average purchase count of  of before and after the usage of each promotion.

In [ ]:
_abtest = ab_test_values['promotion_usage_before_after_amount'].query("is_amount_increased_per_promotions == True")
trace = [go.Bar(name="before avereage order count per customers", x=_abtest['promotions'], y=_abtest['mean_control']),
         go.Bar(name="after avereage order count per customers", x=_abtest['promotions'], y=_abtest['mean_validation'])
         ]
fig = go.Figure(data=trace, layout_yaxis_range=[round(max(0, min(min(_abtest['mean_control']), min(_abtest['mean_validation'])) - 0.02), 2), 
                                               round(max(0, max(max(_abtest['mean_control']), max(_abtest['mean_validation'])) - 0.02), 2)])
fig.update_layout(barmode='group', xaxis_tickangle=-45)

# Change the bar mode
fig.show()

In [ ]:
_abtest.to_csv(join(abspath(""), "exploratory_analysis", "sample_data", "promotion_usage_before_after_amount_accept.csv"), index=False)

## Promotions of Before After Order Count and Payment Amount Differences

In [ ]:
order_amount_ab = pd.merge(ab_test_values['promotion_usage_before_after_orders'], 
         ab_test_values['promotion_usage_before_after_amount'].rename(columns={'diff': 'diff_amount'}), on='promotions', how='inner')[['diff_amount', 'diff', 'promotions']]
order_amount_ab.head()

In [ ]:
import plotly.express as px
fig = px.scatter(order_amount_ab, x="diff", y="diff_amount", color="promotions",
                 )
fig.show()

In [ ]:
order_amount_ab.to_csv(join(abspath(""), "exploratory_analysis", "sample_data", "order_and_payment_amount_differences.csv"), index=False)

### How do promotions work on customers of before - after order count generally?

In [ ]:
mean_control_order = np.mean(ab_test_values['promotion_usage_before_after_orders']['mean_control'])
mean_validation_order = np.mean(ab_test_values['promotion_usage_before_after_orders']['mean_validation'])
mean_control_amount = np.mean(ab_test_values['promotion_usage_before_after_amount']['mean_control'])
mean_validation_amount = np.mean(ab_test_values['promotion_usage_before_after_amount']['mean_validation'])

In [ ]:

trace = [go.Bar(name="before avereage purchase amount per customers", x=['before', 'after'], y=[mean_control_order]),
         go.Bar(name="after avereage purchase amount per customers", x=['after', 'after'], y=[mean_validation_order])
         ]
fig = go.Figure(data=trace)
fig.update_layout(barmode='group', xaxis_tickangle=-45, )
# Change the bar mode
fig.show()



trace = [go.Bar(name="before avereage purchase amount per customers", x=['before', 'after'], y=[mean_control_order, mean_validation_order]),
         go.Bar(name="after avereage purchase amount per customers", x=['before', 'after'], y=[mean_control_amount, mean_validation_amount])
         ]
fig = go.Figure(data=trace)
fig.update_layout(barmode='group', xaxis_tickangle=-45, )
# Change the bar mode
fig.show()

### Comaprison of Promotions
    - Assumption: "Orders with 1st promotion have significantly more effects on Payment Amount then orders with 2nd Promotion of Payments Amount."
    - number of  Promotion Pairs of Comparison: promotion pairs all combinations are tested. The number of 1st Promotion of Accepted tests are counted as 1st promotion has significantly more effects on Payment Amount then 2nd Promotion. 
    - Accept Ratio: Ratio of Accepted the assumption.
        

In [ ]:
_ab_test = ab_test_values['promotion_comparison']
_ab_test.head()

In [ ]:
_ab_test.sort_values('accept_Ratio', ascending=True)

In [ ]:
_ab_test['diff'] = _ab_test['mean_validation'] - _ab_test['mean_control']

In [ ]:
def get_promotion_comparison(x):
    _x = x.split("_")
    return pd.Series(["_".join(_x[0:2]), "_".join(_x[2:4])])

In [ ]:
_ab_test[['1st promo', '2nd Promo']] = _ab_test['promotion_comparison'].apply(lambda x: get_promotion_comparison(x))

In [ ]:
_ab_test['total_positive_effects'] = _ab_test['promo_1st_vs_promo_2nd'].apply(lambda x: 1 if x in ['True', True] else 0)

In [ ]:
_ab_test_pv = _ab_test.groupby("1st promo").agg({"total_effects": "sum", 
                                                 "accept_Ratio": "mean", "promo_1st_vs_promo_2nd": "count"}
                                               ).reset_index().sort_values(['total_effects'], ascending=False).sort_values(
    ['accept_Ratio'], ascending=True).rename(columns={"promo_1st_vs_promo_2nd": "total_negative_effects"})
_ab_test_pv['total_negative_effects'] = _ab_test_pv['total_negative_effects'] - _ab_test_pv['total_effects']
_ab_test_pv

In [ ]:
import plotly.express as px
fig = px.scatter(_ab_test_pv, x="accept_Ratio", y="total_effects", color="1st promo",
                 size='total_negative_effects')
fig.show()

In [ ]:
_ab_test_pv.to_csv(join(abspath(""), "exploratory_analysis", "sample_data", "promotion_comparison.csv"), index=False)

In [ ]:
_ab_test_pv

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=_ab_test_pv['accept_Ratio'], 
                         y=_ab_test_pv['total_effects'], 
                         text=_ab_test_pv['1st promo'],
                         marker=dict(size=_ab_test_pv['total_negative_effects'], color=list(range(len(_ab_test_pv))), colorscale='Rainbow'),
                    mode='markers',
                    name='markers'))

fig.show()

In [ ]:
rfm = pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'rfm'")['data'])[0])

In [ ]:
pd.DataFrame(res).query("report_name == 'product_analytic'")

In [ ]:
pd.DataFrame(list(pd.DataFrame(res).query(
    "report_name == 'product_analytic' and type == 'most_ordered_products'"
)['data'])[0]).sort_values(by='order_count', ascending=False).iloc[:10].to_csv("sample_data_most_ordered_products.csv", index=False)

In [ ]:
pd.DataFrame(list(pd.DataFrame(res).query(
    "report_name == 'product_analytic' and type == 'most_ordered_products'"
)['data'])[0]).sort_values(by='order_count', ascending=False).iloc[:10]# .to_csv("sample_data_most_ordered_categories.csv", index=False)

In [ ]:
pd.DataFrame(list(pd.DataFrame(res).query(
    "report_name == 'cohort' and type == 'customers_journey' and index == 'main'")['data'])[0])

In [ ]:
pd.DataFrame(res)['index'].unique()

In [ ]:
res = []
for r in es.search(index='reports', body={"size": 1000, "from": 0})['hits']['hits']:
    try:
        res.append({'report_name': r['_source']['report_name'],
                    'report_date': r['_source']['report_date'],
                    'time_period': r['_source']['report_types'].get('time_period', None),
                    'type': r['_source']['report_types'].get('type', None),
                    'from': r['_source']['report_types'].get('from', None),
                    'to': r['_source']['report_types'].get('to', None),
                    'abtest_type': r['_source']['report_types'].get('abtest_type', None),
                    'report_types': r['_source']['report_types'],
                    'index': r['_source']['index'],
                    'data': r['_source']['data']})
    except Exception as e:
        print(e)

In [ ]:
pd.DataFrame(res).query("report_name == 'clv_prediction'")

In [ ]:
pd.DataFrame(res).query("index == 'location_1'")['report_name'].unique()

In [ ]:
pd.DataFrame(res).query("index == 'main' and report_name == 'stats'")['type'].unique()

In [ ]:
pd.DataFrame(res)['report_name'].unique()

In [ ]:
pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'rfm'")['data'])[0])

In [ ]:
_res = es.search(index='orders', body={"size": 0, 
                                                               "aggs" : {
        "langs" : {
            "terms" : { "field" : "dimension.keyword",  
                        "size" : 500 }
        }
    }
                               
                               })
_res

In [ ]:
dimension = [r['key'] for r in _res['aggregations']['langs']['buckets']] + ['main']
dimension

In [ ]:
def collect_reports(port, host, index, query=None):
    res = []

    match = {"size": 1000, "from": 0}
    for r in es.search(index='reports', body=match)['hits']['hits']:
        if r['_source']['index'] == index:
            try:
                _obj = {'report_name': r['_source']['report_name'],
                        'report_date': r['_source']['report_date'],
                        'time_period': r['_source']['report_types'].get('time_period', None),
                        'type': r['_source']['report_types'].get('type', None),
                        '_from': r['_source']['report_types'].get('from', None),
                        '_to': r['_source']['report_types'].get('to', None),
                        'abtest_type': r['_source']['report_types'].get('abtest_type', None),
                        'report_types': r['_source']['report_types'],
                        'index': r['_source']['index']
                        }
                if query is not None:
                    _obj['data'] = r['_source']['data']
                res.append(_obj)

            except Exception as e:
                print(e)
    return pd.DataFrame(res)

In [ ]:
es_tag = {"host": 'localhost', "port": 9200}

In [ ]:
double_reports = {'promotion_usage_before_after_amount': ['promotion_usage_before_after_amount_accept',
                                                                       'promotion_usage_before_after_amount_reject',
                                                                       'order_and_payment_amount_differences'],
                               'promotion_usage_before_after_orders': ['promotion_usage_before_after_orders_accept',
                                                                       'promotion_usage_before_after_orders_reject']}

In [ ]:
def report_name(k, c):
    if k['report_name'] == 'funnel':
        r_name = k['time_period'] + '_funnel'
        if k['type'] == 'downloads':
            r_name += '_downloads'
    if k['report_name'] == 'cohort':
        if k['type'] == 'orders':
            r_name = "_".join([k['time_period'], 'cohort_from', str(int(k['_from'])), 'to', str(int(k['_to']))])
        else:
            r_name = "_".join([k['time_period'], 'cohort', k['type']])
    if k['report_name'] == 'stats':
        r_name = k['type'] if k['type'] != '' else 'kpis'
    if k['report_name'] == 'abtest':
        if k['abtest_type'] in list(double_reports.keys()):
            r_name = double_reports[k['abtest_type']]
        else:
            r_name = k['abtest_type']
    if k['report_name'] == 'product_analytic':
        r_name = k['type']
    if k['report_name'] not in ['cohort', 'funnel', 'stats', 'product_analytic', 'abtest']:
        r_name = k['report_name']

    return r_name

In [ ]:
kpis = {}
for index in dimension:
    reports = collect_reports(es_tag['port'], es_tag['host'], index)
    for k in reports.to_dict('results'):
        r_name = report_name(k, index)
        if type(r_name) == list:
            for sub_r_name in r_name:
                kpis[sub_r_name] = {i: k[i] for i in list(k.keys())}
        else:
            kpis[r_name] = {i: k[i] for i in list(k.keys())}

In [ ]:
list(kpis.keys())

In [ ]:
match = {'size': 10000000, 'from': 0, '_source': True, 'query': {'bool': {'must': [{'term': {'index': 'main'}}, {'term': {'report_name': 'stats'}}, {'range': {'report_date': {'gt': '2021-04-22T00:00:00', 'lt': '2021-04-23T18:49:18.337762'}}}]}}}

In [ ]:
pd.DataFrame(res).query("report_name == 'abtest'")

In [ ]:
np.unique([i['abtest_type'] for i in list(pd.DataFrame(res).query("report_name == 'abtest'")['report_types'])]).tolist()

In [ ]:
data = pd.DataFrame(pd.DataFrame(res).query("report_name == 'segmentation'").to_dict('results')[0]['data'])
data.head()

In [ ]:
total_clients = len(data)
data = data.groupby("segments").agg({"client": "count"}).reset_index().rename(columns={"client": "value"})
data['value'] = round((data['value'] * 100) / total_clients, 2)
data

In [ ]:
pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'stats' and type == 'daily_orders'")['data'])[-1]).tail()

In [ ]:
pd.DataFrame(res).query("report_name == 'funnel' and type == 'daily_orders'").to_dict('results')[-1]['data']

In [ ]:
list(pd.DataFrame(res).query("report_name == 'abtest'")['abtest_type'].unique())

In [ ]:
match = {'size': 10000000, 'from': 0, '_source': True, 
         'query': {'bool': {'must': [{'term': {'index': 'main'}}, {'term': {'report_name': 'abtest'}}, 
                                     {'range': {'report_date': {'gt': '2021-04-24T00:00:00', 'lt': '2021-04-25T18:43:02.799896'}}}]}}}

In [ ]:
np.unique([i['_source']['report_types']['abtest_type'] for  i in es.search(index='reports', body=match)['hits']['hits'] if i['_source']['report_name'] == 'abtest'])

In [ ]:
es.search(index='reports', body=match)['hits']['hits']

In [ ]:
list(pd.DataFrame(res).query("report_name == 'product_analytic'")['type'].unique())

In [ ]:
pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'product_analytic' and type == 'hourly_categories_of_sales'")['data'])[-2])

In [ ]:
import tensorflow

In [ ]:
import datetime

def current_date_to_day():
    """
    recent date of converting to datetime from isoformat.
    :return: datetime
    """
    return datetime.datetime.strptime(str(datetime.datetime.now())[0:19], '%Y-%m-%d %H:%M:%S')


def convert_to_date(date):
    """
    Convert str date to datetime. If is NULL skip the process and return None.
    :param date: str format; %Y-%m-%d %H:%M:%S
    :return:
    """
    if date == date:
        return datetime.datetime.strptime(str(date)[0:10] + ' ' + str(date)[11:19], "%Y-%m-%d %H:%M:%S")
    else:
        return None

In [ ]:
with open('/Users/caglanakpinar/Downloads/temporary_folder_for_ca/logs.log') as f:
    file = f.readlines()
file

In [ ]:
user = 'cakpinar23@gmail.com'

In [ ]:
messages = {'ds_connect': "{} Data Source is created or updated", 
 'es_connect': "ElasticSerach Connection is created.",
 'es_delete': "ElasticSerach Connection is deleted.",
 'schedule_start': "Scehduling Process is initialized.",
 'schedule_end': "Scehduling Process is done.",
 'schedule_delete': "Scehduling Process is deleted."}

data_types = {'orders': 'Sessions', 'downloads': 'Customers', 'products': 'Products (Baskets)'}
def create_notification(user, text):
    splits = text.split("-")
    if splits[0] == 'ds':
        message = messages[ds_connect].format(data_types[splits[1].split("_")[0]])
    else:
        message = messages["_".join([splits[0], splits[-1]])]
    
        

In [ ]:
for l in file:
    if l != '\n':
        try:
            if convert_to_date(l[0:19]) > current_date_to_day() - datetime.timedelta(days=7):
                if user in l:
                    print(l)
                    print(l.split(user)[-1])
        except:
            print("can not parse logs")
        


In [ ]:
    
last_7_days_user_logs = [l for l in file if convert_to_date(l[0:19]) > current_date_to_day() - datetime.timedelta(days=7)]
dates

In [ ]:
plot = "sasd*dddd*1111*3333"
_target, _lchart, _index, _date = list(plot.split("*"))
_target

In [ ]:
with open('/Users/caglanakpinar/Downloads/temporary_folder_for_ca/logs.log', 'w') as f:
    f.write("\n".join(file))

In [ ]:
chart_names = {
    "Sessions Of Actions Funnel":
               {"Daily Funnel": "funnel*daily_funnel", 
                "Weekly Funnel": "funnel*weekly_funnel", 
                "Monthly Funnel": "funnel*monthly_funnel", 
                "Hourly Funnel": "funnel*hourly_funnel"}, 
    
    "Customers Of Actions Funnel":
               {"Daily Funnel": "funnel*daily_funnel_downloads", 
                "Weekly Funnel": "funnel*weekly_funnel_downloads", 
                "Monthly Funnel": "funnel*monthly_funnel_downloads", 
                "Hourly Funnel": "funnel*hourly_funnel_downloads"}, 
    "Cohorts":
               {"Daily Download to 1st Order Cohort": "cohort*daily_cohort_downloads", 
                "Daily From 1st to 2nd Order Cohort": "cohort*daily_cohort_from_1_to_2", 
                "Daily From 2nd to 3rd Order Cohort": "cohort*daily_cohort_from_2_to_3", 
                "Daily From 3rd to 4th Order Cohort": "cohort*daily_cohort_from_3_to_4", 
                "Weekly Download to 1st Order Cohort": "cohort*weekly_cohort_downloads",
                "Weekly From 1st to 2nd Order Cohort": "cohort*weekly_cohort_from_1_to_2", 
                "Weekly From 2nd to 3rd Order Cohort": "cohort*weekly_cohort_from_2_to_3", 
                "Weekly From 3rd to 4th Order Cohort": "cohort*weekly_cohort_from_3_to_4"},
    "Descriptive Stats":
               {"Daily Orders": "stats*daily_orders", 
                "Hourly Orders": "stats*hourly_orders", 
                "Weekly Orders": "stats*weekly_orders", 
                "Monthly Orders": "stats*monthly_orders", 
                "Weekly Average Session Count per Customer": "descriptive*weekly_average_session_per_user", 
                "Weekly Average Purchase Count per Customer": "descriptive*weekly_average_payment_amount", 
                "Payment Amount Distribution": "descriptive*purchase_amount_distribution", 
                "Weekly Average Payment Amount": "descriptive*weekly_average_payment_amount"}, 
    "Product Analytics":
               {"Most Combined Products": "product_analytic*most_combined_products", 
                "Most Order Products": "product_analytic*most_ordered_products", 
                "Most Order Categories": "product_analytic*most_ordered_categories"},     
    "A/B test Promotion":
               {"Promotion Comparison": "abtest-promotion*promotion_comparison", 
                "Order And Payment Amount Difference for Before And After Promotion Usage": "abtest-promotion*order_and_payment_amount_differences", 
                "A/B Test Promotion B. - A. Time Periods Cust.s' Avg. Purchase Payment Amount Test (Test Accepted!)": "abtest-promotion*promotion_usage_before_after_amount_accept", 
                "A/B Test Promotion B. - A. Time Periods Cust.s' Avg. Purchase Payment Amount Test (Test Rejected!)": "abtest-promotion*promotion_usage_before_after_amount_reject", 
                "A/B Test Promotion B. - A. Time Periods Cust.s' Tot. Purchase Count Test (Test Accepted!)": "abtest-promotion*promotion_usage_before_after_orders_accept", 
                "A/B Test Promotion B. - A. Time Periods Cust.s' Tott Purchase Count Test (Test Rejected!)": "abtest-promotion*promotion_usage_before_after_orders_reject"}, 
    "A/B Test Product":
               {"A/B Test Product - B. - A. Time Periods Cust.s' Avg. Purchase Payment Amount Test (Test Accepted!)": "abtest-product*product_usage_before_after_amount_accept", 
                "A/B Test Product - B. - A. Time Periods Cust.s' Avg. Purchase Payment Amount Test (Test Rejected!)": "abtest-product*product_usage_before_after_amount_reject", 
                "A/B Test Product - B. - A. Time Periods Cust.s' Totg Purchase Count Test (Test Accepted!)": "abtest-product*product_usage_before_after_orders_accept", 
                "A/B Test Product - B. - A. Time Periods Cust.s' Totg Purchase Count Test (Test Rejected!)": "abtest-product*product_usage_before_after_orders_reject"},
    "A/B Test Customer Segment Change":
               {"A/B Test Cust. Segment Change - A/B Test Product - Daily Customers' Total Order Count per Customer Segment": "abtest-segments*segments_change_daily_before_after_orders", 
                "Weekly Customers' Total Order Count per Customer Segment": "abtest-segments*segments_change_weekly_before_after_orders", 
                "Monthly Customers' Total Order Count per Customer Segment": "abtest-segments*segments_change_weekly_before_after_orders", 
                "Daily Customers' Average Purchase Payment Amount per Customer Segment": "abtest-segments*segments_change_daily_before_after_amount", 
                "Weekly Customers' Average Purchase Payment Amount per Customer Segment": "abtest-segments*segments_change_weekly_before_after_amount", 
                "Monthly Customers' Average Purchase Payment Amount per Customer Segment": "abtest-segments*segments_change_weekly_before_after_amount"},       
    "RFM":
               {"RFM": "rfm*rfm", 
                "Frequency - Recency": "rfm*frequency_recency", 
                "Monetary - Frequency": "rfm*monetary_frequency", 
                "Recency - Monetary": "rfm*recency_monetary"}, 
    "Segmentation":
               {"Customer Segmentation": "customer-segmentation*",
                "Frequency Segmentation": "customer-segmentation*frequency_clusters", 
                "Monetary Segmentation": "customer-segmentation*monetary_clusters", 
                "Recency Segmentation": "customer-segmentation*recency_clusters"}, 
    "CLv Prediction":
               {"Next Week CLV Predictions": "clv*daily_clv", 
                "CLV Predicted Nex Week Customers of Segments of Total Purchase Amounts": "clv*daily_clv"},
    "Overall":
               {"Customer Journey": "index*customer_journey", 
                "Total Number Customer Breakdown with Purchased Order Count": "clv*clvsegments_amount"}}

In [ ]:
from os.path import join
asd = ("adasd", "ssssss")
join(asd[0], asd[1])

In [ ]:
time_diff_str, date_str, prefix_date_str = [""]*3

In [ ]:
time_diff_str

In [ ]:
clv = pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'clv_prediction'")['data'])[0])
clv

In [ ]:
clv.to_csv("/Users/mac/Downloads/es-7.10.0/results.csv", index=False)

In [ ]:
[print("*" * 20) for i in range(3)]

In [ ]:
separator = lambda dim: [print("*" * 20) for i in range(3)] + [print("*"*10, " ", " DIMENSION : ", dim, "*"*10)]

In [ ]:
separator(dim='location_1')

In [ ]:
DATA_WORKS_READABLE_FORM = {'clv_prediction': 'CLv Prediction',
                            'abtest': 'A/B Test', 'funnel': 'Session Actions & Customers Actions Funnels',
                            'cohort': 'Cohorts', 'rfm': 'RFM', 'stats': 'Descriptive Statistics',
                            'products': 'Product Analytics', 'segmentation': 'Customer Segmentation',
                            'anomaly': 'Anomaly Detection'
                            }

In [ ]:
suc_log_for_data_works = lambda x: " {0} is created! Check {0} sections. ".format(
            DATA_WORKS_READABLE_FORM[x])

In [ ]:
try: suc_log_for_data_works('clv_prediction'), suc_log_for_data_works('abtest')
except: print("asdsddd")

In [ ]:
3.0 % 1

In [ ]:
pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'clv_prediction'")['data'])[-1])['client'].unique()

### Anomaly Detection

In [ ]:
daily_funnel_anomaly = pd.DataFrame(list(pd.DataFrame(res).query(
    "report_name == 'anomaly' and type == 'daily_funnel'")['data'])[0])
daily_funnel_anomaly.head()

In [ ]:
min_value = round(min(daily_funnel_anomaly['anomaly_scores']) - 0.01, 2)
max_value = round(max(daily_funnel_anomaly['anomaly_scores']) + 0.01, 2)
daily_funnel_anomaly['outlier'] = daily_funnel_anomaly['outlier'].apply(lambda x: max_value + 0.01 if x == 1 else min_value - 0.01)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    
    go.Scatter(
        name="anomaly score (0 - 1)",
        x=daily_funnel_anomaly['daily'],
        y=daily_funnel_anomaly['anomaly_scores'],
        marker=dict(color="blue")
    ),
    go.Bar(
        name="outlier detection (0/1)",
        x=daily_funnel_anomaly['daily'],
        y=daily_funnel_anomaly['outlier'],
        marker=dict(color='#FECB52')
    )
    
], layout=go.Layout(yaxis={"range": [
                min_value, max_value]})
                )


fig.show()


In [ ]:
daily_funnel_anomaly

### Daily Cohort 

In [ ]:
cohort_anomaly = pd.DataFrame(list(pd.DataFrame(res).query(
    "report_name == 'anomaly' and type == 'cohort_d'")['data'])[0])
cohort_anomaly

In [ ]:
cohort_anomaly_time_line = cohort_anomaly[['daily', 'anomaly_scores_from_d_to_1', 'outlier']
              ].rename(columns={"anomaly_scores_from_d_to_1": "Anomaly Score Download to First Order"})
cohort_anomaly_time_line.head()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    
    go.Scatter(
        name="anomaly score (0 - 1)",
        x=cohort_anomaly_time_line['daily'],
        y=cohort_anomaly_time_line['Anomaly Score Download to First Order'],
        marker=dict(color="blue")
    ),
    go.Bar(
        name="outlier detection (0/1)",
        x=cohort_anomaly_time_line['daily'],
        y=cohort_anomaly_time_line['outlier'],
        marker=dict(color='red')
    )
    
]
                )


fig.show()

In [ ]:
last_20_days = list(cohort_anomaly.query("daily > '2021-05-07'")['daily'])
last_20_days

In [ ]:
last_20_days_cohort = cohort_anomaly.query("daily > '2021-05-07'")[[str(i) for i in list(range(0, 5))]].transpose().reset_index()
last_20_days_cohort.columns = ['days'] + last_20_days
last_20_days_cohort

In [ ]:
outlier_days = list(cohort_anomaly.query("outlier == 1")['daily'])
outlier_days

In [ ]:
cols = []
for col in last_20_days_cohort.columns:
    if col in outlier_days:
        cols.append(col + '_outlier')
    else:
        cols.append(col)
cols

In [ ]:
import plotly.graph_objects as go
x = last_20_days_cohort['days']
fig = go.Figure(data=[
    
    go.Scatter(
        name=col,
        x=x,
        y=last_20_days_cohort[col],
        marker=dict(color="red") if col in outlier_days else dict(color="blue")
    ) for col in last_20_days
    
                    ]
                )


fig.show()

In [ ]:
last_20_days_cohort.columns = cols
last_20_days_cohort

In [ ]:
daily_orders_comparison = pd.DataFrame(list(pd.DataFrame(res).query(
    "report_name == 'anomaly' and type == 'daily_orders_comparison'")['data'])[0])

### Daily Orders Comparison

In [ ]:
columns = ['diff_perc', 'daily']
daily_orders_comparison_normal = daily_orders_comparison.query("anomalities == 'nomal decrease/increase'")[columns]
daily_orders_comparison_decrease = daily_orders_comparison.query("anomalities == 'significant decrease'")[columns]
daily_orders_comparison_increase = daily_orders_comparison.query("anomalities == 'significant increase'")[columns]


In [ ]:
def get_color(metric):
    if metric == 'nomal decrease/increase':
        return 'yellow'
    if metric == 'significant decrease':
        return 'red'
    if metric == 'significant increase':
        return 'green'

In [ ]:
def naming_significance(metric):
    if metric == 'nomal decrease/increase':
        return 'no change'
    if metric == 'significant decrease':
        return 'decrease'
    if metric == 'significant increase':
        return 'increase'

In [ ]:
import plotly.graph_objects as go
x = daily_orders_comparison['daily']
fig = go.Figure(data=[
    
    go.Bar(
        name=col,
        x=x,
        y=df['diff_perc'],
        
    ) for df, col in [(daily_orders_comparison_normal, 'nomal decrease/increase'), 
                  (daily_orders_comparison_decrease, 'significant decrease'), (daily_orders_comparison_increase, 'significant increase')]
    
                    ]
                )


fig.show()

### CLV Prediction Anomaly

In [ ]:
clv_rfm_change = pd.DataFrame(list(pd.DataFrame(res).query("report_name == 'anomaly' and type == 'clv_prediction'")['data'])[0])
clv_rfm_change.head()

In [ ]:
naming_decision = lambda x: 'no change' if x.split(" ")[1] == 'decrease/increase' else x.split(" ")[1]
naming = lambda x1, x2: "Frequency; {0}, Monetary ; {1}".format(naming_decision(x1), naming_decision(x2))

In [ ]:
clv_rfm_change['naming'] = clv_rfm_change.apply(lambda row: naming(row['f_anomaly'], row['m_anomaly']), axis=1)
clv_rfm_change.head()

In [ ]:
clusters = sorted(list(clv_rfm_change['naming'].unique()))
clusters

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=[go.Scatter(name=cluster,
                                 x=clv_rfm_change.query("naming == @cluster")['monetary_diff'], 
                                 y=clv_rfm_change.query("naming == @cluster")['frequency_diff'], 
                                 mode='markers') for cluster in clusters]
                )


fig.show()

### results

In [ ]:
pd.concat([daily_orders_comparison_normal,daily_orders_comparison_decrease, 
           daily_orders_comparison_increase])

In [ ]:
daily_funnel_anomaly = daily_funnel_anomaly.rename(columns={"anomaly_scores":"Anomaly Score Download to First Order"})
daily_funnel_anomaly.head()

In [ ]:
daily_orders_comparison_normal['anomalities'] = 'no change'
daily_orders_comparison_decrease['anomalities'] = 'decrease'
daily_orders_comparison_increase['anomalities'] = 'increase'


In [ ]:
pd.concat([daily_orders_comparison_normal,daily_orders_comparison_decrease, 
           daily_orders_comparison_increase]).to_csv("exploratory_analysis/sample_data/sample_data_dorders_anomaly.csv", index=False)

In [ ]:
daily_funnel_anomaly.to_csv("exploratory_analysis/sample_data/sample_data_dfunnel_anomaly.csv", index=False)

In [ ]:
daily_funnel_anomaly.to_csv("exploratory_analysis/sample_data/sample_data_dfunnel_anomaly.csv", index=False)
last_20_days_cohort.to_csv("exploratory_analysis/sample_data/sample_data_dcohort_anomaly.csv", index=False)
cohort_anomaly_time_line.to_csv("exploratory_analysis/sample_data/sample_data_dcohort_anomaly_2.csv", index=False)
pd.concat([daily_orders_comparison_normal,daily_orders_comparison_decrease, 
           daily_orders_comparison_increase]).to_csv("exploratory_analysis/sample_data/sample_data_dorders_anomaly.csv", index=False)
clv_rfm_change

In [ ]:
clv_rfm_change.to_csv("exploratory_analysis/sample_data/sample_data_clvrfm_anomaly.csv", index=False)

In [ ]:
pd.DataFrame(res).query(
    "report_name == 'anomaly'  and type == 'daily_orders_comparison' ")